In [1]:
import requests
import time
import pandas as pd
import random
from lxml import html
import lmdbm
import lzma
import pickle


In [2]:
http_cache = lmdbm.open("wnv_http_cache.lmdb", flag="c")
def cache_get(url):
    if url not in http_cache:
        http_cache[url] = lzma.compress(pickle.dumps(requests.get(url).text))
        time.sleep(1)
        http_cache.sync()
    return pickle.loads(lzma.decompress(http_cache[url]))

In [3]:

def get_counties_for_year(year):
    text = cache_get(f"http://www.idph.state.il.us/envhealth/wnvsurveillance_data_{year:02}.htm")
    root = html.fromstring(text)
    return set([e.text.strip() for e in root.xpath("//td[@width='475']//table//td[position()=1]/font/a")])

def get_all_counties():
    counties = set()
    for year in range(2, 16):
        counties |= get_counties_for_year(year)
    return counties

all_counties = get_all_counties()


In [4]:

def get_tables(logs):
    human_tables = []
    animal_tables = []

    for county in all_counties:
        if "cook" in county.lower():
            continue
        for year in range(2, 17):
            def emit(m):
                print(f"{county} {year}: {m}")
                logs.append((county, year, m))
            emit("start")
            try:
                clean_county_name = county.lower().replace(".", "").replace(" ", "")
                table_page_text = cache_get(f"http://www.idph.state.il.us/envhealth/wnvcounty/wnv{clean_county_name}{year:02}.htm")
            except Exception as e:
                emit(("Failed to download ", e))
                continue
            try:
                table_page_root = html.fromstring(table_page_text)
            except Exception as e:
                emit(("Failed to parse ", e))
                continue
            try:
                tables_in_detail_page_animal = table_page_root.xpath("//table[not(.//table) and .//*[contains(text(), 'Municipality')]]")
                tables_in_detail_page_human = table_page_root.xpath("//table[not(.//table) and .//*[contains(text(), 'Date Reported')]]")

            except Exception as e:
                emit(("Failed to extract ", e))
                continue
            
            for human_table in tables_in_detail_page_human:
                human_table = html.tostring(human_table)
                human_table = pd.read_html(human_table, header=0)[0]
                human_tables.append(human_table)
                emit("has humans")
            for animal_table in  tables_in_detail_page_animal:
                animal_table = html.tostring(animal_table)
                animal_table = pd.read_html(animal_table, header=0)[0]
                animal_tables.append(animal_table)
                emit("has animals")
            if not tables_in_detail_page_human and not tables_in_detail_page_animal:
                emit("no huamn and animals tables")
            
            emit("success")
    
    return pd.concat(human_tables) if human_tables else None, pd.concat(animal_tables) if animal_tables else None


In [5]:
logs = []
human_tables, animal_tables = get_tables(logs)

Gallatin 2: start
Gallatin 2: has animals
Gallatin 2: success
Gallatin 3: start
Gallatin 3: no huamn and animals tables
Gallatin 3: success
Gallatin 4: start
Gallatin 4: no huamn and animals tables
Gallatin 4: success
Gallatin 5: start
Gallatin 5: has animals
Gallatin 5: success
Gallatin 6: start
Gallatin 6: has animals
Gallatin 6: success
Gallatin 7: start
Gallatin 7: has animals
Gallatin 7: success
Gallatin 8: start
Gallatin 8: has animals
Gallatin 8: success
Gallatin 9: start
Gallatin 9: has animals
Gallatin 9: success
Gallatin 10: start
Gallatin 10: has animals
Gallatin 10: success
Gallatin 11: start
Gallatin 11: has animals
Gallatin 11: success
Gallatin 12: start
Gallatin 12: no huamn and animals tables
Gallatin 12: success
Gallatin 13: start
Gallatin 13: has animals
Gallatin 13: success
Gallatin 14: start
Gallatin 14: has animals
Gallatin 14: success
Gallatin 15: start
Gallatin 15: has animals
Gallatin 15: success
Gallatin 16: start
Gallatin 16: has animals
Gallatin 16: success
R

Clinton 16: has animals
Clinton 16: success
Jackson 2: start
Jackson 2: has humans
Jackson 2: has animals
Jackson 2: success
Jackson 3: start
Jackson 3: has animals
Jackson 3: success
Jackson 4: start
Jackson 4: has humans
Jackson 4: has animals
Jackson 4: success
Jackson 5: start
Jackson 5: has animals
Jackson 5: success
Jackson 6: start
Jackson 6: has animals
Jackson 6: success
Jackson 7: start
Jackson 7: has animals
Jackson 7: success
Jackson 8: start
Jackson 8: has animals
Jackson 8: success
Jackson 9: start
Jackson 9: has animals
Jackson 9: success
Jackson 10: start
Jackson 10: has animals
Jackson 10: success
Jackson 11: start
Jackson 11: has animals
Jackson 11: success
Jackson 12: start
Jackson 12: has animals
Jackson 12: success
Jackson 13: start
Jackson 13: has animals
Jackson 13: success
Jackson 14: start
Jackson 14: has animals
Jackson 14: success
Jackson 15: start
Jackson 15: has animals
Jackson 15: success
Jackson 16: start
Jackson 16: has animals
Jackson 16: success
Will 2

Whiteside 2: has animals
Whiteside 2: success
Whiteside 3: start
Whiteside 3: has humans
Whiteside 3: has animals
Whiteside 3: success
Whiteside 4: start
Whiteside 4: has animals
Whiteside 4: success
Whiteside 5: start
Whiteside 5: has animals
Whiteside 5: success
Whiteside 6: start
Whiteside 6: has animals
Whiteside 6: success
Whiteside 7: start
Whiteside 7: has animals
Whiteside 7: success
Whiteside 8: start
Whiteside 8: has animals
Whiteside 8: success
Whiteside 9: start
Whiteside 9: no huamn and animals tables
Whiteside 9: success
Whiteside 10: start
Whiteside 10: has animals
Whiteside 10: success
Whiteside 11: start
Whiteside 11: no huamn and animals tables
Whiteside 11: success
Whiteside 12: start
Whiteside 12: has animals
Whiteside 12: success
Whiteside 13: start
Whiteside 13: has animals
Whiteside 13: success
Whiteside 14: start
Whiteside 14: has animals
Whiteside 14: success
Whiteside 15: start
Whiteside 15: has animals
Whiteside 15: success
Whiteside 16: start
Whiteside 16: h

Kane 14: has animals
Kane 14: success
Kane 15: start
Kane 15: has animals
Kane 15: success
Kane 16: start
Kane 16: has animals
Kane 16: success
McHenry 2: start
McHenry 2: has humans
McHenry 2: has animals
McHenry 2: success
McHenry 3: start
McHenry 3: has animals
McHenry 3: success
McHenry 4: start
McHenry 4: has animals
McHenry 4: success
McHenry 5: start
McHenry 5: has humans
McHenry 5: has animals
McHenry 5: success
McHenry 6: start
McHenry 6: has animals
McHenry 6: success
McHenry 7: start
McHenry 7: has animals
McHenry 7: success
McHenry 8: start
McHenry 8: has animals
McHenry 8: success
McHenry 9: start
McHenry 9: no huamn and animals tables
McHenry 9: success
McHenry 10: start
McHenry 10: has animals
McHenry 10: success
McHenry 11: start
McHenry 11: has animals
McHenry 11: success
McHenry 12: start
McHenry 12: has animals
McHenry 12: success
McHenry 13: start
McHenry 13: has animals
McHenry 13: success
McHenry 14: start
McHenry 14: has animals
McHenry 14: success
McHenry 15: st

Grundy 13: has animals
Grundy 13: success
Grundy 14: start
Grundy 14: has animals
Grundy 14: success
Grundy 15: start
Grundy 15: has animals
Grundy 15: success
Grundy 16: start
Grundy 16: has animals
Grundy 16: success
Cass 2: start
Cass 2: has animals
Cass 2: success
Cass 3: start
Cass 3: has animals
Cass 3: success
Cass 4: start
Cass 4: has animals
Cass 4: success
Cass 5: start
Cass 5: no huamn and animals tables
Cass 5: success
Cass 6: start
Cass 6: no huamn and animals tables
Cass 6: success
Cass 7: start
Cass 7: no huamn and animals tables
Cass 7: success
Cass 8: start
Cass 8: no huamn and animals tables
Cass 8: success
Cass 9: start
Cass 9: no huamn and animals tables
Cass 9: success
Cass 10: start
Cass 10: no huamn and animals tables
Cass 10: success
Cass 11: start
Cass 11: no huamn and animals tables
Cass 11: success
Cass 12: start
Cass 12: no huamn and animals tables
Cass 12: success
Cass 13: start
Cass 13: no huamn and animals tables
Cass 13: success
Cass 14: start
Cass 14: n

Vermilion 13: no huamn and animals tables
Vermilion 13: success
Vermilion 14: start
Vermilion 14: no huamn and animals tables
Vermilion 14: success
Vermilion 15: start
Vermilion 15: has animals
Vermilion 15: success
Vermilion 16: start
Vermilion 16: has animals
Vermilion 16: success
Fayette 2: start
Fayette 2: has animals
Fayette 2: success
Fayette 3: start
Fayette 3: has animals
Fayette 3: success
Fayette 4: start
Fayette 4: has animals
Fayette 4: success
Fayette 5: start
Fayette 5: no huamn and animals tables
Fayette 5: success
Fayette 6: start
Fayette 6: no huamn and animals tables
Fayette 6: success
Fayette 7: start
Fayette 7: no huamn and animals tables
Fayette 7: success
Fayette 8: start
Fayette 8: no huamn and animals tables
Fayette 8: success
Fayette 9: start
Fayette 9: no huamn and animals tables
Fayette 9: success
Fayette 10: start
Fayette 10: no huamn and animals tables
Fayette 10: success
Fayette 11: start
Fayette 11: no huamn and animals tables
Fayette 11: success
Fayette 

Lawrence 3: has animals
Lawrence 3: success
Lawrence 4: start
Lawrence 4: no huamn and animals tables
Lawrence 4: success
Lawrence 5: start
Lawrence 5: no huamn and animals tables
Lawrence 5: success
Lawrence 6: start
Lawrence 6: has animals
Lawrence 6: success
Lawrence 7: start
Lawrence 7: no huamn and animals tables
Lawrence 7: success
Lawrence 8: start
Lawrence 8: no huamn and animals tables
Lawrence 8: success
Lawrence 9: start
Lawrence 9: no huamn and animals tables
Lawrence 9: success
Lawrence 10: start
Lawrence 10: no huamn and animals tables
Lawrence 10: success
Lawrence 11: start
Lawrence 11: no huamn and animals tables
Lawrence 11: success
Lawrence 12: start
Lawrence 12: no huamn and animals tables
Lawrence 12: success
Lawrence 13: start
Lawrence 13: no huamn and animals tables
Lawrence 13: success
Lawrence 14: start
Lawrence 14: no huamn and animals tables
Lawrence 14: success
Lawrence 15: start
Lawrence 15: no huamn and animals tables
Lawrence 15: success
Lawrence 16: start

In [6]:
http_cache.sync()

In [7]:
text = requests.get("http://www.idph.state.il.us/envhealth/wnvsurveillance_data_09.htm").text
root = html.fromstring(text)
[e.text for e in root.xpath("//td[@width='475']//table//td[position()=1]/font/a")]

[' Adams',
 'Bond',
 ' Bureau',
 ' Champaign',
 ' Clinton',
 ' Cook',
 ' DuPage',
 ' Edgar',
 ' Franklin',
 ' Gallatin',
 ' Grundy',
 ' Jackson',
 ' Jersey',
 ' Kane',
 ' Kendall',
 ' Knox',
 ' Lake',
 ' LaSalle',
 ' Macon',
 ' Macoupin',
 ' Madison',
 ' Marion',
 ' Massac',
 ' Mercer',
 ' Ogle',
 ' Perry',
 ' St. Clair',
 ' Sangamon',
 ' Stephenson',
 ' Tazewell',
 ' Warren',
 ' Washington',
 ' Wayne',
 ' Will',
 ' Williamson',
 ' Winnebago']

In [8]:
human_tables.to_csv("/Users/ericliao/Desktop/human_tables.csv")
animal_tables.to_csv("/Users/ericliao/Desktop/animal_tables.csv")